In [1]:
import pandas as pd
import os
import time
import numpy as np
import torch

In [2]:
import tensorflow as tf
#physical_devices = tf.config.list_physical_devices('CPU')
physical_devices = tf.config.list_physical_devices('GPU')

/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.12) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


In [4]:
#tf.config.experimental.set_memory_growth(physical_devices[0], enable=True)
physical_devices

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [5]:
#test
x = torch.rand(5, 3)
print(x)

tensor([[0.2754, 0.5066, 0.6013],
        [0.1235, 0.8745, 0.1099],
        [0.4657, 0.0304, 0.7356],
        [0.7233, 0.4006, 0.7177],
        [0.7838, 0.2859, 0.2795]])


In [6]:
#test
torch.cuda.is_available()

True

In [7]:
#! pip install transformers
#! conda install pytorch torchvision torchaudio cudatoolkit=10.1 -c pytorch
#! pip3 install torch==1.7.0 torchvision==0.8.1 -f https://download.pytorch.org/whl/cu101/torch_stable.html
#! pip install tensorflow
! nvidia-smi

Thu Nov 24 14:38:33 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.103.01   Driver Version: 470.103.01   CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-PCI...  Off  | 00000000:61:00.0 Off |                    0 |
| N/A   28C    P0    33W / 250W |      3MiB / 40536MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# Logits Extractor
>
> Extracting Tensor Logits from a given Neural Code Model @danaderp
>

In [8]:
def params(): 
    return {
            'big_table_path' : '/workspaces/CausalCodeCapability/data/big_table/testbed_concepts.csv',
            'hf_model' : 'EleutherAI/gpt-neo-1.3B' 
}

In [9]:
#pwd
parameters = params()
parameters['big_table_path']

'/workspaces/CausalCodeCapability/data/big_table/testbed_concepts.csv'

# Data Upload

In [10]:
data_pd = pd.read_csv( parameters['big_table_path'] )

In [11]:
data_pd.head(5)

,Unnamed: 0,max_method_length,whole_func_string,ast_concepts,model_tokenizer_concepts
0,0,200,"def get_vid_from_url(url):\n """"""Extract...",[['def' 'def' 'function_definition']\n ['get_v...,[['def' 'def' 'function_definition']\n [' get'...
1,1,200,"def sina_xml_to_url_list(xml_data):\n """"""st...",[['def' 'def' 'function_definition']\n ['sina_...,[['def' 'def' 'function_definition']\n [' s' '...
2,2,200,"def makeMimi(upid):\n """"""From http://cdn37....",[['def' 'def' 'function_definition']\n ['makeM...,[['def' 'def' 'function_definition']\n [' make...
3,3,200,"def fc2video_download(url, output_dir = '.', m...",[['def' 'def' 'function_definition']\n ['fc2vi...,[['def' 'def' 'function_definition']\n [' f' '...
4,4,200,"def dailymotion_download(url, output_dir='.', ...",[['def' 'def' 'function_definition']\n ['daily...,[['def' 'def' 'function_definition']\n [' dail...


In [11]:
data_pd.ast_concepts.values[0]

'[[\'def\' \'def\' \'function_definition\']\n [\'get_vid_from_url\' \'identifier\' \'function_definition\']\n [\'(\' \'(\' \'parameters\']\n [\'url\' \'identifier\' \'parameters\']\n [\')\' \')\' \'parameters\']\n [\':\' \':\' \'function_definition\']\n [\'"""Extracts video ID from URL.\\n        """\' \'string\'\n  \'expression_statement\']\n [\'return\' \'return\' \'return_statement\']\n [\'match1\' \'identifier\' \'call\']\n [\'(\' \'(\' \'argument_list\']\n [\'url\' \'identifier\' \'argument_list\']\n [\',\' \',\' \'argument_list\']\n ["r\'youtu\\\\.be/([^?/]+)\'" \'string\' \'argument_list\']\n [\')\' \')\' \'argument_list\']\n [\'or\' \'or\' \'boolean_operator\']\n [\'match1\' \'identifier\' \'call\']\n [\'(\' \'(\' \'argument_list\']\n [\'url\' \'identifier\' \'argument_list\']\n [\',\' \',\' \'argument_list\']\n ["r\'youtube\\\\.com/embed/([^/?]+)\'" \'string\' \'argument_list\']\n [\')\' \')\' \'argument_list\']\n [\'or\' \'or\' \'boolean_operator\']\n [\'match1\' \'identifier

In [12]:
data_pd.whole_func_string.values[0]

'def get_vid_from_url(url):\n        """Extracts video ID from URL.\n        """\n        return match1(url, r\'youtu\\.be/([^?/]+)\') or \\\n          match1(url, r\'youtube\\.com/embed/([^/?]+)\') or \\\n          match1(url, r\'youtube\\.com/v/([^/?]+)\') or \\\n          match1(url, r\'youtube\\.com/watch/([^/?]+)\') or \\\n          parse_query_param(url, \'v\') or \\\n          parse_query_param(parse_query_param(url, \'u\'), \'v\')'

In [13]:
data_pd.whole_func_string.values[0][:100]

'def get_vid_from_url(url):\n        """Extracts video ID from URL.\n        """\n        return match1('

# Model Upload

In [14]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import pipeline

In [14]:
#pip list
#! pip install git+https://github.com/huggingface/transfomers.git


In [12]:
#tokenizer = AutoTokenizer.from_pretrained("Salesforce/codegen-2B-mono")
#tokenizer = AutoTokenizer.from_pretrained("EleutherAI/gpt-j-6B")
#generator = pipeline('text-generation', model='EleutherAI/gpt-neo-1.3B')
#model = AutoModelForCausalLM.from_pretrained("EleutherAI/gpt-j-6B")
generator = pipeline('text-generation', model='EleutherAI/gpt-neo-125M')


In [16]:
generator("EleutherAI has", do_sample=True, min_length=20)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'EleutherAI has released an article about how the government agencies and their staff are being unfairly unfairly targeted'}]

In [24]:
generator( data_pd.whole_func_string.values[0][:100], do_sample=True, min_length=20 )


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Input length of input_ids is 49, but `max_length` is set to 20. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.


[{'generated_text': 'def get_vid_from_url(url):\n        """Extracts video ID from URL.\n        """\n        return match1(url'}]

In [13]:
generator2 = pipeline('text-generation', model='EleutherAI/gpt-neo-1.3B')


In [26]:
generator2("for(int i= ", do_sample=True, min_length=50)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'for(int i=  0, k=0; i<m; i++)\n        {\n          if (m[i]!= i)\n   '}]

In [28]:
generator2( data_pd.whole_func_string.values[0][:100], do_sample=True, max_new_tokens=50 )

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'def get_vid_from_url(url):\n        """Extracts video ID from URL.\n        """\n        return match1(url, re.compile("([0-9]+)", re.IGNORECASE)["name"])\n\n\ndef delete_vid_from_url(url):\n        """Deletes video'}]

# Extracting Logits From a Given Model

In [15]:
from transformers import GPTNeoForCausalLM, GPT2Tokenizer
model = GPTNeoForCausalLM.from_pretrained( parameters['hf_model'] )
tokenizer = GPT2Tokenizer.from_pretrained( parameters['hf_model'] )

In [16]:
tokenizer.get_vocab()

{'!': 0,
 '"': 1,
 '#': 2,
 '$': 3,
 '%': 4,
 '&': 5,
 "'": 6,
 '(': 7,
 ')': 8,
 '*': 9,
 '+': 10,
 ',': 11,
 '-': 12,
 '.': 13,
 '/': 14,
 '0': 15,
 '1': 16,
 '2': 17,
 '3': 18,
 '4': 19,
 '5': 20,
 '6': 21,
 '7': 22,
 '8': 23,
 '9': 24,
 ':': 25,
 ';': 26,
 '<': 27,
 '=': 28,
 '>': 29,
 '?': 30,
 '@': 31,
 'A': 32,
 'B': 33,
 'C': 34,
 'D': 35,
 'E': 36,
 'F': 37,
 'G': 38,
 'H': 39,
 'I': 40,
 'J': 41,
 'K': 42,
 'L': 43,
 'M': 44,
 'N': 45,
 'O': 46,
 'P': 47,
 'Q': 48,
 'R': 49,
 'S': 50,
 'T': 51,
 'U': 52,
 'V': 53,
 'W': 54,
 'X': 55,
 'Y': 56,
 'Z': 57,
 '[': 58,
 '\\': 59,
 ']': 60,
 '^': 61,
 '_': 62,
 '`': 63,
 'a': 64,
 'b': 65,
 'c': 66,
 'd': 67,
 'e': 68,
 'f': 69,
 'g': 70,
 'h': 71,
 'i': 72,
 'j': 73,
 'k': 74,
 'l': 75,
 'm': 76,
 'n': 77,
 'o': 78,
 'p': 79,
 'q': 80,
 'r': 81,
 's': 82,
 't': 83,
 'u': 84,
 'v': 85,
 'w': 86,
 'x': 87,
 'y': 88,
 'z': 89,
 '{': 90,
 '|': 91,
 '}': 92,
 '~': 93,
 '¡': 94,
 '¢': 95,
 '£': 96,
 '¤': 97,
 '¥': 98,
 '¦': 99,
 '§': 100

In [17]:
len( tokenizer.get_vocab() ) #todo an assert

50257

In [20]:
#prompts = data_pd.whole_func_string.values[:2]
prompts = data_pd.whole_func_string.values

In [22]:
prompts[:2]

array(['def get_vid_from_url(url):\n        """Extracts video ID from URL.\n        """\n        return match1(url, r\'youtu\\.be/([^?/]+)\') or \\\n          match1(url, r\'youtube\\.com/embed/([^/?]+)\') or \\\n          match1(url, r\'youtube\\.com/v/([^/?]+)\') or \\\n          match1(url, r\'youtube\\.com/watch/([^/?]+)\') or \\\n          parse_query_param(url, \'v\') or \\\n          parse_query_param(parse_query_param(url, \'u\'), \'v\')',
       'def sina_xml_to_url_list(xml_data):\n    """str->list\n    Convert XML to URL List.\n    From Biligrab.\n    """\n    rawurl = []\n    dom = parseString(xml_data)\n    for node in dom.getElementsByTagName(\'durl\'):\n        url = node.getElementsByTagName(\'url\')[0]\n        rawurl.append(url.childNodes[0].data)\n    return rawurl'],
      dtype=object)

In [21]:
#tokenized_code = tokenizer(prompt, return_tensors="pt")
#input_ids = tokenized_code.input_ids #tst

In [23]:
#Making space for the tensors
input_ids_list = tokenizer.batch_encode_plus( prompts )
input_ids_list = [torch.Tensor( np.array( input_ids ) ) for input_ids in input_ids_list.input_ids]

Token indices sequence length is longer than the specified maximum sequence length for this model (2197 > 2048). Running this sequence through the model will result in indexing errors


In [24]:
tokenizer.decode( input_ids_list[0] ) #Decoding IDs

'def get_vid_from_url(url):\n        """Extracts video ID from URL.\n        """\n        return match1(url, r\'youtu\\.be/([^?/]+)\') or \\\n          match1(url, r\'youtube\\.com/embed/([^/?]+)\') or \\\n          match1(url, r\'youtube\\.com/v/([^/?]+)\') or \\\n          match1(url, r\'youtube\\.com/watch/([^/?]+)\') or \\\n          parse_query_param(url, \'v\') or \\\n          parse_query_param(parse_query_param(url, \'u\'), \'v\')'

In [25]:
#Output is the class CausalLMOutputWithPast (https://huggingface.co/transformers/v4.10.1/main_classes/output.html?highlight=causallmoutputwithpast)
#logits (torch.FloatTensor of shape (batch_size, sequence_length, config.vocab_size)) – Prediction scores of the language modeling head (scores for each vocabulary token before SoftMax).
# output = model(input_ids=input_ids)
# [WARNING TAKES TIME]
output = [ model( input_ids = i.to(torch.int64) ) for i in input_ids_list ]

In [169]:
output_logits = [ o.logits for o in output ]

In [172]:
#Based on the response here: https://stackoverflow.com/questions/62852940/how-to-get-immediate-next-word-probability-using-gpt2-model
# next_token_logits = output[0][:, -1, :] #last token
output_logits[1].shape #The loggits contains the information of the next token prediction given the prevous windows w_{<t}

torch.Size([143, 50257])

In [176]:
output_logits[1][0]

tensor([ -4.7163,  -4.1678,  -8.0573,  ..., -14.3003, -12.5503, -10.1957],
       grad_fn=<SelectBackward0>)

In [177]:
sum( output_logits[1][0] ) #non-normalized tokens summup different than 1

tensor(-605830.6875, grad_fn=<AddBackward0>)

In [178]:
soft = torch.nn.Softmax( dim = 0 ) #Flattening normalization
next_token_distribution = soft( output_logits[1][0] ) #Flattening normalization

In [179]:
next_token_distribution.shape

torch.Size([50257])

## Batch Softmax Distribution

In [183]:
output_logits

[tensor([[ -4.7163,  -4.1678,  -8.0573,  ..., -14.3003, -12.5503, -10.1957],
         [ -9.1358,  -7.5095,  -9.0157,  ..., -19.7905, -16.6783, -11.4492],
         [-10.6732,  -8.5573,  -7.7642,  ..., -15.8004, -16.4903, -12.3207],
         ...,
         [ -7.8279,  -8.3893,  -6.1925,  ..., -16.1534, -17.7415, -13.3287],
         [-11.4834, -15.5278,  -6.6332,  ..., -21.3085, -19.0391, -11.0297],
         [-11.9949, -10.6260, -10.0660,  ..., -24.9658, -19.0741,  -7.5699]],
        grad_fn=<MmBackward0>),
 tensor([[ -4.7163,  -4.1678,  -8.0573,  ..., -14.3003, -12.5503, -10.1957],
         [ -5.9883,  -3.5874,  -5.9038,  ..., -16.7872, -17.1196,  -9.3145],
         [ -9.2882,  -7.0703,  -7.5206,  ..., -22.5321, -17.8830, -12.0654],
         ...,
         [-13.4025, -10.5316, -14.8570,  ..., -21.6318, -19.1788, -12.2755],
         [-17.7037, -16.9761, -15.4234,  ..., -29.2865, -25.8262, -16.9394],
         [-12.0943, -12.7120, -10.5151,  ..., -25.6224, -25.2612,  -8.3305]],
        grad_f

In [184]:
next_token_distributions = [ [soft( logits ) for logits in prompt_tesor]  for prompt_tesor in output_logits ]

In [185]:
sum( next_token_distributions[0][0] ) #tst

tensor(1.0000, grad_fn=<AddBackward0>)

In [188]:
def topk_tuple( vocab_token, largest):
    token_position_dict = list( tokenizer.get_vocab().keys() )
    topk = vocab_token.topk( k=1 , largest=largest ) #TODO K number of elements can be extended
    return ( token_position_dict[topk.indices], topk.values)

In [189]:
#maximum case
#return A namedtuple of (values, indices)
max_logit_token_prompt = [[ topk_tuple(vocab_token=vocab_token, largest=True) for vocab_token in prompt ] for prompt in next_token_distributions ]
max_logit_token_prompt

[[('module', tensor([0.3343], grad_fn=<TopkBackward0>)),
  ('_', tensor([0.5017], grad_fn=<TopkBackward0>)),
  ('all', tensor([0.0101], grad_fn=<TopkBackward0>)),
  ('_', tensor([0.4963], grad_fn=<TopkBackward0>)),
  ('id', tensor([0.0770], grad_fn=<TopkBackward0>)),
  ('_', tensor([0.9812], grad_fn=<TopkBackward0>)),
  ('id', tensor([0.0433], grad_fn=<TopkBackward0>)),
  ('(', tensor([0.7238], grad_fn=<TopkBackward0>)),
  ('url', tensor([0.6483], grad_fn=<TopkBackward0>)),
  (')', tensor([0.3734], grad_fn=<TopkBackward0>)),
  ('Ċ', tensor([0.9387], grad_fn=<TopkBackward0>)),
  ('Ġ', tensor([0.8314], grad_fn=<TopkBackward0>)),
  ('Ġ', tensor([0.9674], grad_fn=<TopkBackward0>)),
  ('Ġ', tensor([0.9959], grad_fn=<TopkBackward0>)),
  ('Ġ', tensor([0.1247], grad_fn=<TopkBackward0>)),
  ('Ġ', tensor([0.9800], grad_fn=<TopkBackward0>)),
  ('Ġ', tensor([0.9704], grad_fn=<TopkBackward0>)),
  ('Ġ', tensor([0.9949], grad_fn=<TopkBackward0>)),
  ('Ġtry', tensor([0.1583], grad_fn=<TopkBackward0>))

In [190]:
#minimum case
min_logit_token_prompt = [[ topk_tuple(vocab_token=vocab_token, largest=False) for vocab_token in prompt ] for prompt in next_token_distributions ]

In [191]:
min_logit_token_prompt

[[('ĠPremiership', tensor([8.5841e-14], grad_fn=<TopkBackward0>)),
  ('ĠEthiop', tensor([7.7133e-14], grad_fn=<TopkBackward0>)),
  ('ĠUnderworld', tensor([1.0263e-17], grad_fn=<TopkBackward0>)),
  ('Ġacknow', tensor([2.0740e-16], grad_fn=<TopkBackward0>)),
  ('ĠElven', tensor([6.0369e-19], grad_fn=<TopkBackward0>)),
  ('ĠMinotaur', tensor([1.1095e-18], grad_fn=<TopkBackward0>)),
  ('ĠElven', tensor([3.7817e-18], grad_fn=<TopkBackward0>)),
  ('isSpecial', tensor([5.3122e-17], grad_fn=<TopkBackward0>)),
  ('ĠMinotaur', tensor([8.4047e-18], grad_fn=<TopkBackward0>)),
  ('ĠScion', tensor([1.5539e-18], grad_fn=<TopkBackward0>)),
  ('ĠNASL', tensor([1.7348e-19], grad_fn=<TopkBackward0>)),
  ('ĠNASL', tensor([4.1314e-20], grad_fn=<TopkBackward0>)),
  ('uphem', tensor([8.8674e-18], grad_fn=<TopkBackward0>)),
  ('ĠSetFontSize', tensor([3.9860e-19], grad_fn=<TopkBackward0>)),
  ('Ġundrafted', tensor([3.8516e-16], grad_fn=<TopkBackward0>)),
  ('ĠNASL', tensor([4.3423e-19], grad_fn=<TopkBackward0>

In [ ]:
#actual token case

actual_logit_token_prompt = [[ ( vocab_token[] ) for vocab_token in prompt ] for prompt in next_token_distributions ]

In [223]:
#actual token case
actual_logit_token_prompt = []
token_position_dict = list( tokenizer.get_vocab().keys() )
for p,prompt in enumerate( input_ids_list ):
    temp_prompt = []
    for c,code_token_position in enumerate( prompt[1:] ): #Eliminate the first token prediction since we do not use it
        temp_prompt.append( 
                ( token_position_dict[int(code_token_position)],next_token_distributions[ p ][ c ][int(code_token_position)] )        
            )
    actual_logit_token_prompt.append( temp_prompt )

In [224]:
actual_logit_token_prompt

[[('Ġget', tensor(0.0052, grad_fn=<SelectBackward0>)),
  ('_', tensor(0.5017, grad_fn=<SelectBackward0>)),
  ('vid', tensor(4.7143e-05, grad_fn=<SelectBackward0>)),
  ('_', tensor(0.4963, grad_fn=<SelectBackward0>)),
  ('from', tensor(0.0489, grad_fn=<SelectBackward0>)),
  ('_', tensor(0.9812, grad_fn=<SelectBackward0>)),
  ('url', tensor(0.0405, grad_fn=<SelectBackward0>)),
  ('(', tensor(0.7238, grad_fn=<SelectBackward0>)),
  ('url', tensor(0.6483, grad_fn=<SelectBackward0>)),
  ('):', tensor(0.2895, grad_fn=<SelectBackward0>)),
  ('Ċ', tensor(0.9387, grad_fn=<SelectBackward0>)),
  ('Ġ', tensor(0.8314, grad_fn=<SelectBackward0>)),
  ('Ġ', tensor(0.9674, grad_fn=<SelectBackward0>)),
  ('Ġ', tensor(0.9959, grad_fn=<SelectBackward0>)),
  ('Ġ', tensor(0.1247, grad_fn=<SelectBackward0>)),
  ('Ġ', tensor(0.9800, grad_fn=<SelectBackward0>)),
  ('Ġ', tensor(0.9704, grad_fn=<SelectBackward0>)),
  ('Ġ', tensor(0.9949, grad_fn=<SelectBackward0>)),
  ('Ġ"""', tensor(0.0722, grad_fn=<SelectBackwa

In [198]:
len(input_ids_list[0][:-1])

211

In [208]:
temp = [0,1,2,3]
temp[:-1]
temp[1:]

[1, 2, 3]

In [204]:
input_ids_list[0][:-1]

tensor([4.2990e+03, 6.5100e+02, 6.2000e+01, 1.6921e+04, 6.2000e+01, 6.7380e+03,
        6.2000e+01, 6.3710e+03, 7.0000e+00, 6.3710e+03, 2.5990e+03, 1.9800e+02,
        2.2000e+02, 2.2000e+02, 2.2000e+02, 2.2000e+02, 2.2000e+02, 2.2000e+02,
        2.2000e+02, 3.7227e+04, 1.1627e+04, 9.7400e+02, 8.2000e+01, 2.0080e+03,
        4.5220e+03, 4.2200e+02, 1.0289e+04, 1.3000e+01, 1.9800e+02, 2.2000e+02,
        2.2000e+02, 2.2000e+02, 2.2000e+02, 2.2000e+02, 2.2000e+02, 2.2000e+02,
        3.7227e+04, 1.9800e+02, 2.2000e+02, 2.2000e+02, 2.2000e+02, 2.2000e+02,
        2.2000e+02, 2.2000e+02, 2.2000e+02, 1.4410e+03, 2.8720e+03, 1.6000e+01,
        7.0000e+00, 6.3710e+03, 1.1000e+01, 3.7400e+02, 6.0000e+00, 3.2594e+04,
        1.7405e+04, 1.3500e+03, 1.4000e+01, 2.6933e+04, 6.1000e+01, 3.0000e+01,
        1.4000e+01, 6.0000e+01, 2.8988e+04, 1.1537e+04, 3.9300e+02, 3.4670e+03,
        1.9800e+02, 2.2000e+02, 2.2000e+02, 2.2000e+02, 2.2000e+02, 2.2000e+02,
        2.2000e+02, 2.2000e+02, 2.2000e+

In [205]:
len(input_ids_list[0])

212

In [92]:
token_position_dict = list( tokenizer.get_vocab().keys() )

In [94]:
token_position_dict[ max_logit_token.indices ]

'module'

In [109]:
# Actual Token Probability
actual_token = tokenizer.decode( input_ids[0][1] ) #Decoding IDs
actual_token

' get'

## Batching for Experiments

In [48]:
gen_tokens = model.generate(input_ids, do_sample=True, temperature=0.9, max_length=200,)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [55]:
gen_tokens

tensor([[ 4299,   651,    62, 32604,    62,  1676,  1443,     7,  7568,    25,
           279,    67,    13,  6601, 19778,    11,  2746,    25,  9104,    11,
           299,    25, 32233,    58,   600,    60,   796,  6045,  2599,   220,
           220,   220,   220,  3497,   262,  1612, 12867,   286,  1123, 11241,
           326,   262,  4981,    71,   426,  4331,   329,   281,  2104, 19798,
           292,  1366, 14535,    13,   361,   299,   318,  6045,    25,    77,
           796, 18896,     7,  7568,     8,     2,  9058,  9290,  8341,   329,
           262,  1271,   286, 40279,   290,  2160,   286, 39522,   329,  1123,
         11241,  9127,    82,   796,   685,    15,    60,  1635,  2746,    13,
         30001,  7509,    13,  1136,    62, 18893,   397,    62,  7857,  3419,
         16345,    62,  1676,  1443,   796,   685,    15,    13,    15,    60,
          1635,  2746,    13, 30001,  7509,    13,  1136,    62, 18893,   397,
            62,  7857,  3419,  7783,   954,    82,  

In [49]:
gen_text = tokenizer.batch_decode(gen_tokens)[0]

In [50]:
gen_text

'def get_mean_probs(df: pd.DataFrame, model: Model, n: Optional[int] = None):     Get the mean probability of each token that the modelshould predict for an entire pandas dataframe.if n is None:n = len(df)# setup container lists for the number of occurrences and sum of probabilities for each tokencounts = [0] * model.tokenizer.get_vocab_size()sum_probs = [0.0] * model.tokenizer.get_vocab_size()return counts + sum_probs# count the number of model predictions if needed\n\ndef get_model_probs(df: pd.DataFrame, model: Model): get_model_probs_counts, get_mean_probs = get_means, get_means_probs(model)\n\n# build the mean'